In [1]:
import argparse
import sys
import re
import os, glob, datetime, time
import numpy as np
import torch
import torch.nn as nn
from torch.nn.modules.loss import _Loss
import torch.nn.init as init
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import data_generator as dg
from data_generator import DenoisingDataset
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# Params
if any(["jupyter" in arg for arg in sys.argv]):
    # Simulate command line arguments (replace these with your desired defaults)
    sys.argv = ['ipykernel_launcher.py', '--model', 'DnCNN', '--batch_size', '64', '--train_data', 'data/Train400', '--sigma', '4', '--epoch', '3', '--lr', '0.0005']

parser = argparse.ArgumentParser(description='PyTorch DnCNN')
parser.add_argument('--model', default='DnCNN', type=str, help='choose a type of model')
parser.add_argument('--batch_size', default=64, type=int, help='batch size')
parser.add_argument('--train_data', default='data/Train400', type=str, help='path of train data')
parser.add_argument('--sigma', default=4, type=int, help='noise level')
parser.add_argument('--epoch', default=3, type=int, help='number of train epoches')
parser.add_argument('--lr', default=5e-4, type=float, help='initial learning rate for Adam')
args = parser.parse_args()
batch_size = args.batch_size
cuda = torch.cuda.is_available()
n_epoch = args.epoch
sigma = args.sigma

save_dir = os.path.join('models', args.model+'_' + 'sigma' + str(sigma))

if not os.path.exists(save_dir):
    os.mkdir(save_dir)
        
        
class DnCNN(nn.Module):
    def __init__(self, out_channels=64, in_channels=1, kernel_size=3):
        super(DnCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1, stride=1)
        self.BN = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        output_layers = []
        for _ in range(10):
            for sub in self.make_Residual():
                output_layers.append(sub)
        self.out = nn.Sequential(*output_layers)
        self.res = nn.Conv2d(64, 1, kernel_size=3, padding=1, stride=1)
        
    def make_Residual(self, in_channels=64, out_channels=64, kernel_size=3):
        i = 0
        layers = []
        for i in range(2):
            layers.append(nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=1, bias=True))
            layers.append(nn.BatchNorm2d(64, eps=0.0001, momentum=0.95))
            layers.append(nn.ReLU(inplace=True))
        return layers

    
    def forward(self, x):
        y = x
        out = self.relu(self.BN(self.conv1(x)))
        out = self.out(out)
        dn = y - self.res(out)
        return dn
        

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                print('init weight')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

                
class sum_squared_error(_Loss):  # PyTorch 0.4.1
    """
    Definition: sum_squared_error = 1/2 * nn.MSELoss(reduction = 'sum')
    The backward is defined as: input-target
    """
    def __init__(self, size_average=None, reduce=None, reduction='sum'):
        super(sum_squared_error, self).__init__(size_average, reduce, reduction)

    def forward(self, input, target):
        # return torch.sum(torch.pow(input-target,2), (0,1,2,3)).div_(2)
        return torch.nn.functional.mse_loss(input, target, size_average=None, reduce=None, reduction='sum').div_(2)


def log(*args, **kwargs):
     print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S:"), *args, **kwargs)

In [4]:
import torch

# Attempt to manually select a CUDA device
try:
    torch.cuda.current_device()
    print("Current CUDA device:", torch.cuda.get_device_name())
except Exception as e:
    print("Error accessing CUDA device:", str(e))

Current CUDA device: NVIDIA GeForce RTX 2080 Ti


In [5]:
from tqdm import tqdm

if __name__ == '__main__':
    # model selection
    print('===> Building model')
    model = DnCNN()
    
    initial_epoch = 0
    model.train()
    # criterion = nn.MSELoss(reduction = 'sum')  # PyTorch 0.4.1
    criterion = sum_squared_error()
    if cuda:
        model = model.cuda()
         # device_ids = [0]
         # model = nn.DataParallel(model, device_ids=device_ids).cuda()
         # criterion = criterion.cuda()
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay = 1e-4)
    scheduler = MultiStepLR(optimizer, milestones=[3, 6, 9], gamma=0.2)  # learning rates
    for epoch in tqdm(range(initial_epoch, n_epoch)):

        scheduler.step(epoch)  # step to the learning rate in this epcoh
        xs = dg.datagenerator(data_dir=args.train_data)
        xs = xs.astype('float32')/255.0
        xs = torch.from_numpy(xs.transpose((0, 3, 1, 2)))  # tensor of the clean patches, NXCXHXW
        DDataset = DenoisingDataset(xs, sigma)
        DLoader = DataLoader(dataset=DDataset, num_workers=4, drop_last=True, batch_size=batch_size, shuffle=True)
        epoch_loss = 0
        start_time = time.time()

        for n_count, batch_yx in enumerate(DLoader):
                optimizer.zero_grad()
                if cuda:
                    batch_x, batch_y = batch_yx[1].cuda(), batch_yx[0].cuda()
                loss = criterion(model(batch_y), batch_x)
                epoch_loss += loss.item()
                loss.backward()
                optimizer.step()
                if n_count % 10 == 0:
                    print('%4d %4d / %4d loss = %2.4f' % (epoch+1, n_count, xs.size(0)//batch_size, loss.item()/batch_size))
        elapsed_time = time.time() - start_time

        log('epcoh = %4d , loss = %4.4f , time = %4.2f s' % (epoch+1, epoch_loss/n_count, elapsed_time))
        np.savetxt('train_result.txt', np.hstack((epoch+1, epoch_loss/n_count, elapsed_time)), fmt='%2.4f')
        # torch.save(model.state_dict(), os.path.join(save_dir, 'model_%03d.pth' % (epoch+1)))
        torch.save(model, os.path.join(save_dir, 'model_%03d.pth' % (epoch+1)))


===> Building model


  0%|          | 0/3 [00:00<?, ?it/s]

^_^-training data finished-^_^
   1    0 / 2728 loss = 74.5793
   1   10 / 2728 loss = 1.9732
   1   20 / 2728 loss = 0.6997
   1   30 / 2728 loss = 0.3605
   1   40 / 2728 loss = 0.3070
   1   50 / 2728 loss = 0.2544
   1   60 / 2728 loss = 0.2318
   1   70 / 2728 loss = 0.2427
   1   80 / 2728 loss = 0.2478
   1   90 / 2728 loss = 0.2374
   1  100 / 2728 loss = 0.2324
   1  110 / 2728 loss = 0.2304
   1  120 / 2728 loss = 0.2278
   1  130 / 2728 loss = 0.2258
   1  140 / 2728 loss = 0.2255
   1  150 / 2728 loss = 0.2160
   1  160 / 2728 loss = 0.2117
   1  170 / 2728 loss = 0.2230
   1  180 / 2728 loss = 0.2067
   1  190 / 2728 loss = 0.2268
   1  200 / 2728 loss = 0.2130
   1  210 / 2728 loss = 0.2591
   1  220 / 2728 loss = 0.2136
   1  230 / 2728 loss = 0.2114
   1  240 / 2728 loss = 0.2137
   1  250 / 2728 loss = 0.2098
   1  260 / 2728 loss = 0.2109
   1  270 / 2728 loss = 0.2019
   1  280 / 2728 loss = 0.2141
   1  290 / 2728 loss = 0.2111
   1  300 / 2728 loss = 0.2072
   1  3

 33%|███▎      | 1/3 [02:50<05:41, 170.89s/it]

2024-11-22 05:30:45: epcoh =    1 , loss = 20.1438 , time = 167.67 s
^_^-training data finished-^_^
   2    0 / 2728 loss = 0.1363
   2   10 / 2728 loss = 0.1408
   2   20 / 2728 loss = 0.1503
   2   30 / 2728 loss = 0.1477
   2   40 / 2728 loss = 0.1470
   2   50 / 2728 loss = 0.1379
   2   60 / 2728 loss = 0.1486
   2   70 / 2728 loss = 0.1421
   2   80 / 2728 loss = 0.1565
   2   90 / 2728 loss = 0.1583
   2  100 / 2728 loss = 0.1507
   2  110 / 2728 loss = 0.1543
   2  120 / 2728 loss = 0.1396
   2  130 / 2728 loss = 0.1387
   2  140 / 2728 loss = 0.1513
   2  150 / 2728 loss = 0.1439
   2  160 / 2728 loss = 0.1414
   2  170 / 2728 loss = 0.1535
   2  180 / 2728 loss = 0.1446
   2  190 / 2728 loss = 0.1373
   2  200 / 2728 loss = 0.1417
   2  210 / 2728 loss = 0.1522
   2  220 / 2728 loss = 0.1427
   2  230 / 2728 loss = 0.2112
   2  240 / 2728 loss = 0.1562
   2  250 / 2728 loss = 0.1395
   2  260 / 2728 loss = 0.1416
   2  270 / 2728 loss = 0.1420
   2  280 / 2728 loss = 0.1362
 

 67%|██████▋   | 2/3 [05:43<02:51, 171.91s/it]

2024-11-22 05:33:37: epcoh =    2 , loss = 8.8900 , time = 169.31 s
^_^-training data finished-^_^
   3    0 / 2728 loss = 0.1288
   3   10 / 2728 loss = 0.1159
   3   20 / 2728 loss = 0.1302
   3   30 / 2728 loss = 0.1170
   3   40 / 2728 loss = 0.1335
   3   50 / 2728 loss = 0.1268
   3   60 / 2728 loss = 0.1257
   3   70 / 2728 loss = 0.1201
   3   80 / 2728 loss = 0.1244
   3   90 / 2728 loss = 0.1313
   3  100 / 2728 loss = 0.1308
   3  110 / 2728 loss = 0.1331
   3  120 / 2728 loss = 0.1223
   3  130 / 2728 loss = 0.1187
   3  140 / 2728 loss = 0.1236
   3  150 / 2728 loss = 0.1199
   3  160 / 2728 loss = 0.1215
   3  170 / 2728 loss = 0.1164
   3  180 / 2728 loss = 0.1383
   3  190 / 2728 loss = 0.1195
   3  200 / 2728 loss = 0.1197
   3  210 / 2728 loss = 0.1191
   3  220 / 2728 loss = 0.1230
   3  230 / 2728 loss = 0.1176
   3  240 / 2728 loss = 0.1257
   3  250 / 2728 loss = 0.1346
   3  260 / 2728 loss = 0.1346
   3  270 / 2728 loss = 0.1376
   3  280 / 2728 loss = 0.1434
  

100%|██████████| 3/3 [08:35<00:00, 171.86s/it]

2024-11-22 05:36:29: epcoh =    3 , loss = 7.9540 , time = 168.78 s
